## 1. 算法设计

研究出发点：

轨迹密度与通行速度间的关系：密集的轨迹可能意味着交通拥堵，从而影响速度。

路段属性与行驶时间的关系：如路段类型、最高限速、是否有桥梁或隧道等因素可能影响行驶时间。

节假日与工作日的交通模式差异：节假日可能有不同的交通流量和速度模式。

## 2. 实现算法

In [3]:
import pandas as pd

base_dir = './data/'
road_df = pd.read_csv(base_dir + 'road.csv')

In [5]:
road_df.head()

,id,coordinates,highway,length,lanes,tunnel,bridge,maxspeed,width,alley,roundabout
0,0,"[[116.3894407, 39.9062721], [116.3894463, 39.9...",4,28.981,0,0,0,0,0,0,0
1,1,"[[116.3894407, 39.9062721], [116.386428, 39.90...",4,325.040,5,0,0,0,0,0,0
2,2,"[[116.3930703, 39.906394], [116.3894407, 39.90...",4,309.891,5,0,0,0,0,0,0
3,3,"[[116.3970962, 39.9065222], [116.3930703, 39.9...",4,343.693,5,0,0,0,0,0,0
4,4,"[[116.3970962, 39.9065222], [116.3970144, 39.9...",11,34.704,0,0,0,0,0,0,0


In [6]:
import pandas as pd

# 加载数据
traj_df = pd.read_csv('./data/traj.csv')
eta_task_df = pd.read_csv('./data/eta_task.csv')

In [7]:
traj_df.head()

,id,time,entity_id,traj_id,coordinates,current_dis,speeds,holidays
0,0,2013-10-08T17:45:00Z,254,0,"[116.318726,40.009014]",0.000000,36.6900,0
1,1,2013-10-08T17:46:45Z,254,0,"[116.315102,40.004784]",0.562623,24.5375,0
2,2,2013-10-08T17:47:39Z,254,0,"[116.315018,40.002842]",0.778695,31.9675,0
3,3,2013-10-08T17:49:26Z,254,0,"[116.315041,39.998585]",1.252148,19.7850,0
4,4,2013-10-08T17:51:15Z,254,0,"[116.315605,39.992554]",1.924533,24.4500,0


In [8]:
eta_task_df.head()

,id,traj_id,time,entity_id,coordinates,current_dis,speeds,holidays
0,126,9,2013-10-08T22:30:00Z,254,"[116.490013,39.892902]",0.000000,37.2925,0
1,133,9,NaN,254,"[116.511292,39.904655]",3.079017,24.6500,0
2,147,11,2013-10-08T23:00:00Z,254,"[116.450546,39.943771]",0.000000,36.0575,0
3,164,11,NaN,254,"[116.440742,40.018944]",12.543428,49.9000,0
4,171,13,2013-10-08T17:15:00Z,255,"[116.335503,39.763187]",0.000000,26.6250,0


In [9]:
import pandas as pd

# 加载数据
traj_df = pd.read_csv('./data/traj.csv')
eta_task_df = pd.read_csv('./data/eta_task.csv')
jump_task_df = pd.read_csv('./data/jump_task.csv')

In [10]:
eta_task_df.head()

,id,traj_id,time,entity_id,coordinates,current_dis,speeds,holidays
0,126,9,2013-10-08T22:30:00Z,254,"[116.490013,39.892902]",0.000000,37.2925,0
1,133,9,NaN,254,"[116.511292,39.904655]",3.079017,24.6500,0
2,147,11,2013-10-08T23:00:00Z,254,"[116.450546,39.943771]",0.000000,36.0575,0
3,164,11,NaN,254,"[116.440742,40.018944]",12.543428,49.9000,0
4,171,13,2013-10-08T17:15:00Z,255,"[116.335503,39.763187]",0.000000,26.6250,0


In [11]:
eta_task_df = eta_task_df.dropna()

In [12]:
eta_task_df['start_time'] = pd.to_datetime(eta_task_df.groupby('entity_id')['time'].transform('min'))
eta_task_df['end_time'] = pd.to_datetime(eta_task_df.groupby('entity_id')['time'].transform('max'))
eta_task_df['ETA'] = (eta_task_df['end_time'] - eta_task_df['start_time']).dt.total_seconds()

In [13]:
# 假设第一个和最后一个坐标分别是出发地点和目标地点
eta_task_df['start_coordinates'] = eta_task_df.groupby('entity_id')['coordinates'].transform('first')
eta_task_df['end_coordinates'] = eta_task_df.groupby('entity_id')['coordinates'].transform('last')

In [14]:
eta_task_df.head()

,id,traj_id,time,entity_id,coordinates,current_dis,speeds,holidays,start_time,end_time,ETA,start_coordinates,end_coordinates
0,126,9,2013-10-08T22:30:00Z,254,"[116.490013,39.892902]",0.0,37.2925,0,2013-10-08 22:30:00+00:00,2013-10-30 20:15:00+00:00,1892700.0,"[116.490013,39.892902]","[116.531364,40.020645]"
2,147,11,2013-10-08T23:00:00Z,254,"[116.450546,39.943771]",0.0,36.0575,0,2013-10-08 22:30:00+00:00,2013-10-30 20:15:00+00:00,1892700.0,"[116.490013,39.892902]","[116.531364,40.020645]"
4,171,13,2013-10-08T17:15:00Z,255,"[116.335503,39.763187]",0.0,26.6250,0,2013-10-08 17:15:00+00:00,2013-10-30 00:15:00+00:00,1839600.0,"[116.335503,39.763187]","[116.308769,39.869987]"
6,207,16,2013-10-08T19:00:00Z,255,"[116.479919,39.801834]",0.0,22.7600,0,2013-10-08 17:15:00+00:00,2013-10-30 00:15:00+00:00,1839600.0,"[116.335503,39.763187]","[116.308769,39.869987]"
8,337,24,2013-10-08T08:00:00Z,256,"[116.229218,39.900791]",0.0,26.2850,0,2013-10-08 08:00:00+00:00,2013-10-30 16:00:00+00:00,1929600.0,"[116.229218,39.900791]","[116.312134,39.87331]"


In [15]:
eta_task_df_v1 = eta_task_df.drop('id',1).drop('time',1).drop('coordinates',1)
eta_task_df_v1['start_lon'] = eta_task_df_v1['start_coordinates'].apply(lambda x:eval(x)[0])
eta_task_df_v1['start_lat'] = eta_task_df_v1['start_coordinates'].apply(lambda x:eval(x)[1])
eta_task_df_v1['end_lon'] = eta_task_df_v1['end_coordinates'].apply(lambda x:eval(x)[0])
eta_task_df_v1['end_lat'] = eta_task_df_v1['end_coordinates'].apply(lambda x:eval(x)[1])
eta_task_df_v1 = eta_task_df_v1.drop('start_coordinates',1).drop('end_coordinates',1)

In [16]:
eta_task_df_v1.head()

,traj_id,entity_id,current_dis,speeds,holidays,start_time,end_time,ETA,start_lon,start_lat,end_lon,end_lat
0,9,254,0.0,37.2925,0,2013-10-08 22:30:00+00:00,2013-10-30 20:15:00+00:00,1892700.0,116.490013,39.892902,116.531364,40.020645
2,11,254,0.0,36.0575,0,2013-10-08 22:30:00+00:00,2013-10-30 20:15:00+00:00,1892700.0,116.490013,39.892902,116.531364,40.020645
4,13,255,0.0,26.6250,0,2013-10-08 17:15:00+00:00,2013-10-30 00:15:00+00:00,1839600.0,116.335503,39.763187,116.308769,39.869987
6,16,255,0.0,22.7600,0,2013-10-08 17:15:00+00:00,2013-10-30 00:15:00+00:00,1839600.0,116.335503,39.763187,116.308769,39.869987
8,24,256,0.0,26.2850,0,2013-10-08 08:00:00+00:00,2013-10-30 16:00:00+00:00,1929600.0,116.229218,39.900791,116.312134,39.873310


In [17]:
eta_task_df_v1['trip_duration'] = (eta_task_df_v1['end_time'] - eta_task_df_v1['start_time']).dt.total_seconds()

In [18]:
list(eta_task_df_v1.columns)

['traj_id',
 'entity_id',
 'current_dis',
 'speeds',
 'holidays',
 'start_time',
 'end_time',
 'ETA',
 'start_lon',
 'start_lat',
 'end_lon',
 'end_lat',
 'trip_duration']

In [19]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import numpy as np

# 特征选择
features = ['traj_id','entity_id','current_dis','speeds','holidays',
            'start_lon','start_lat', 'end_lon', 'end_lat', 'trip_duration']
target = 'ETA'

# 分割特征和目标
X = eta_task_df_v1[features]
y = eta_task_df_v1[target]

# 数据集划分
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 模型训练
model = LinearRegression()
model.fit(X_train, y_train)

# 预测
predictions = model.predict(X_test)

In [20]:
from sklearn.metrics import mean_squared_error

# 计算评估指标
mse = mean_squared_error(y_test, predictions)
rmse = np.sqrt(mse)

print(f'MSE: {mse}, RMSE: {rmse}')

MSE: 3.8080387472441204e-21, RMSE: 6.170930843271638e-11


In [21]:
X_test.head()

,traj_id,entity_id,current_dis,speeds,holidays,start_lon,start_lat,end_lon,end_lat,trip_duration
548,1367,4136,0.0,25.8575,0,116.357285,39.918407,116.457970,39.910744,1611900.0
6860,17396,4269,0.0,29.6225,1,116.439697,39.956863,116.461372,39.948837,1856700.0
8648,21689,4351,0.0,54.0000,0,116.296432,39.971817,116.211220,39.901512,1920600.0
2054,5136,4119,0.0,27.3325,1,116.455490,39.950119,116.428665,39.933636,1781100.0
3178,8007,255,0.0,36.4675,0,116.335503,39.763187,116.308769,39.869987,1839600.0


In [22]:
y_test.head()

548     1611900.0
6860    1856700.0
8648    1920600.0
2054    1781100.0
3178    1839600.0
Name: ETA, dtype: float64

In [24]:
X_test['ETA'] = y_test

/Users/apple/anaconda3/envs/dp/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [25]:
X_test.head()

,traj_id,entity_id,current_dis,speeds,holidays,start_lon,start_lat,end_lon,end_lat,trip_duration,ETA
548,1367,4136,0.0,25.8575,0,116.357285,39.918407,116.457970,39.910744,1611900.0,1611900.0
6860,17396,4269,0.0,29.6225,1,116.439697,39.956863,116.461372,39.948837,1856700.0,1856700.0
8648,21689,4351,0.0,54.0000,0,116.296432,39.971817,116.211220,39.901512,1920600.0,1920600.0
2054,5136,4119,0.0,27.3325,1,116.455490,39.950119,116.428665,39.933636,1781100.0,1781100.0
3178,8007,255,0.0,36.4675,0,116.335503,39.763187,116.308769,39.869987,1839600.0,1839600.0


In [26]:
X_test.to_csv('test_3.csv',encoding='utf-8',index=False)